In [ ]:
from matplotlib import pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df=pd.read_csv('drive/My Drive/churn-7043.csv')

In [ ]:
df=df.drop(['customerID'], axis=1)
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)


churn=cat_df['churn']
cat_df=cat_df.drop(['churn'], axis=1)


num_df=num_df.fillna(conData)

result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df

X=result_df_op
y=churn



In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
train=pd.concat([X_train, y_train], axis=1)
test=pd.concat([X_test, y_test], axis=1)
#train=pd.concat([X, y], axis=1)
#test=pd.concat([X, y], axis=1)
X_train=X
X_test=X
y_train=y
y_test=y

In [ ]:
from google.colab import files
import os, sys
file_path = 'drive/My Drive/Colab Notebooks/private-data-generation/'
#sys.path.append(file_path)
sys.path.append(os.path.abspath(file_path))
import sys
sys.path.insert(1,'/content/drive/My Drive/Colab Notebooks/private-data-generation/')

In [ ]:
from models import dp_wgan, pate_gan, ron_gauss

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn import preprocessing
from scipy.special import expit
from models import dp_wgan, pate_gan, ron_gauss
from models.Private_PGM import private_pgm
import argparse
import numpy as np
import pandas as pd
import collections
import os

In [ ]:
class  Class_opt:
    "This is a person class"
    categorical = False
    model='dp-wgan'
    target_epsilon=80
    target_delta=0.8
    downstream_task="classification"
    target_variable='status'
    batch_size=64
    micro_batch_size=8
    clamp_lower=0.01
    clamp_upper=0.01
    clip_coeff=0.1
    sigma=2.0
    num_epochs=50
    enable_privacy=0




opt = Class_opt()
opt.target_variable='churn'
opt.categorical = True
opt.model='dp-wgan'
opt.target_epsilon=10
opt.target_delta=0.1
opt.batch_size=64
opt.micro_batch_size=8
opt.clamp_lower=0.01
opt.clamp_upper=0.01
opt.clip_coeff=0.1
opt.sigma=0.8
opt.num_epochs=200
opt.enable_privacy=1

opt.downstream_task="classification"

In [ ]:
data_columns = [col for col in train.columns if col != opt.target_variable]

In [ ]:
class_ratios = train[opt.target_variable].sort_values().groupby(train[opt.target_variable]).size().values/train.shape[0]
class_ratios

array([0.73732252, 0.26267748])

In [ ]:
X_train = np.nan_to_num(train.drop([opt.target_variable], axis=1).values)
y_train = np.nan_to_num(train[opt.target_variable].values)
X_test = np.nan_to_num(test.drop([opt.target_variable], axis=1).values)
y_test = np.nan_to_num(test[opt.target_variable].values)

In [ ]:
#Normalized the data
X_train = expit(X_train)
X_test = expit(X_test)

In [ ]:
input_dim = X_train.shape[1]
z_dim = int(input_dim / 4 + 1) if input_dim % 4 == 0 else int(input_dim / 4)

Hyperparams = collections.namedtuple('Hyperarams','batch_size micro_batch_size clamp_lower clamp_upper clip_coeff sigma class_ratios lr num_epochs')
Hyperparams.__new__.__defaults__ = (None, None, None, None, None, None, None, None, None)

In [ ]:
conditional = (opt.downstream_task == "classification")
model = dp_wgan.DP_WGAN(input_dim, z_dim, opt.target_epsilon, opt.target_delta, conditional)
model.train(X_train, y_train, Hyperparams(batch_size=opt.batch_size, micro_batch_size=opt.micro_batch_size,
                                              clamp_lower=opt.clamp_lower, clamp_upper=opt.clamp_upper,
                                              clip_coeff=opt.clip_coeff, sigma=opt.sigma, class_ratios=class_ratios, lr=
                                              5e-5, num_epochs=opt.num_epochs), private=opt.enable_privacy)

torch.save(model, 'drive/My Drive/chkp_GAN_MWOE_privacy_1_70-vs-30_dataset_7043.pth')

Epoch : 1 Loss D real :  0.023633150927306217 Loss D fake :  0.011200472013079383 Loss G :  0.011138412393702984 Epsilon spent :  3.1167599694076045
Epoch : 2 Loss D real :  0.023387797516278196 Loss D fake :  0.011132085482600971 Loss G :  0.011028241995057825 Epsilon spent :  3.3552885725726513
Epoch : 3 Loss D real :  0.023553350454735493 Loss D fake :  0.010966809334206701 Loss G :  0.010995611210031703 Epsilon spent :  3.5938171757376987
Epoch : 4 Loss D real :  0.023073368957931033 Loss D fake :  0.010978057440792992 Loss G :  0.011119151195983093 Epsilon spent :  3.8323457789027455
Epoch : 5 Loss D real :  0.023029385717418846 Loss D fake :  0.011032644192692915 Loss G :  0.011016501331500301 Epsilon spent :  4.070874382067792
Epoch : 6 Loss D real :  0.024315279535886 Loss D fake :  0.010868105753987747 Loss G :  0.011147934656146177 Epsilon spent :  4.30940298523284
Epoch : 7 Loss D real :  0.022383515267714267 Loss D fake :  0.010893426292089874 Loss G :  0.011052493538391097

In [ ]:
if opt.model == 'imle' or opt.model == 'dp-wgan' or opt.model == 'pate-gan':
    syn_data = model.generate(X_train.shape[0], class_ratios)
    X_syn, y_syn = syn_data[:, :-1], syn_data[:, -1]

In [ ]:
X_syn_df = pd.DataFrame(data=X_syn, columns=data_columns)
y_syn_df = pd.DataFrame(data=y_syn, columns=[opt.target_variable])

syn_df = pd.concat([X_syn_df, y_syn_df], axis=1)
syn_df.to_csv("drive/My Drive/synthetic_data_GAN_mWOE_privacy_1_70persent_training_dataset7043.csv")
test.to_csv("drive/My Drive/Original_data_GAN_mWOE_privacy_1_30percent_testing_dataset7043.csv")
print("Saved synthetic data at : ", "drive/My Drive/synthetic_data_GAN_mWOE_privacy_1_70persent_training_dataset7043.csv")

Saved synthetic data at :  drive/My Drive/synthetic_data_GAN_mWOE_privacy_1_70persent_training_dataset7043.csv


In [ ]:
# Creating downstream learners
learners = []
if opt.downstream_task == "classification":
    names = ['LR', 'Random Forest', 'Neural Network', 'GaussianNB', 'GradientBoostingClassifier', 'DT', 'KNN', 'SVN']

    learners.append((LogisticRegression()))
    learners.append((RandomForestClassifier()))
    learners.append((MLPClassifier(early_stopping=True)))
    learners.append((GaussianNB()))
    learners.append((GradientBoostingClassifier()))
    learners.append(DecisionTreeClassifier())
    learners.append((KNeighborsClassifier()))
    #learners.append((svm.SVC()))

    print("AUC scores of downstream classifiers on test data : ")
    for i in range(0, len(learners)):
        score = learners[i].fit(X_syn, y_syn)
        #score = learners[i].fit(X_train, y_train)
        pred_probs = learners[i].predict_proba(X_test)
        auc_score = roc_auc_score(y_test, pred_probs[:, 1])
        print('-' * 40)
        print('{0}: {1}'.format(names[i], auc_score))

AUC scores of downstream classifiers on test data : 
----------------------------------------
LR: 0.49990660934178266
----------------------------------------
Random Forest: 0.604261896837849
----------------------------------------
Neural Network: 0.5067286554235635
----------------------------------------
GaussianNB: 0.5954644968337737
----------------------------------------
GradientBoostingClassifier: 0.36816918085638667
----------------------------------------
DT: 0.5
----------------------------------------
KNN: 0.5


In [ ]:
import numpy as np
class WOE_Encoder():
    def __init__(self, cols=None, size=None):
            self.cols = cols
            self.min_samples=1
            #self.bins=10000
            self.bins=int(size/10)
            self._mapping = {}

    def WOE_fit(self, X, y):
        for col in self.cols:
            X[col]=X[col].fillna(-9999)
            if (len(np.unique(X[col]))>100):
                binned_x = pd.qcut(X[col], self.bins,  duplicates='drop')
                d0 = pd.DataFrame({'x': binned_x, 'y':y})
            else:
                d0 = pd.DataFrame({'x': X[col], 'y': y})
            #print (d0)
            # Share of positive (resp. negative) labels for each category P(X=X_i | Y=1) (resp. P(X=X_i | Y=0))
            #mapping = y.groupby(X[col]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping = y.groupby(d0["x"]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping['neg'] = mapping['count'] - mapping['pos']
            mapping[['pos', 'neg']] /= mapping[['pos', 'neg']].sum()
            # For corner cases, defaulting to WOE = 0 (meaning no info). To avoid division by 0 we use default values.
            undef = (mapping['count'] < self.min_samples) | (mapping['pos'] == 0) | (mapping['neg'] == 0)
            mapping.loc[undef, ['pos', 'neg']] = -1
            # Final step, log of ratio of probabily estimates
            mapping['value'] = np.log((mapping['pos'] +0.0001)/ (mapping['neg']+0.0001))
            self._mapping[col] = mapping


        X_encoded = X.copy(deep=True)
        for col, mapping in self._mapping.items():
            X_encoded.loc[:, col] = X_encoded[col].fillna(-9999).map(mapping['value'])
            X_encoded[col].fillna(0, inplace=True)

        return X_encoded

In [ ]:
syn_df=pd.read_csv('drive/My Drive/synthetic_data_GAN_mWOE_privacy_1_70persent_training_dataset7043.csv')
y_syn_df=syn_df['churn']
X_syn_df=syn_df.drop(['churn'], axis=1)
Obj = WOE_Encoder(cols=X_syn_df.columns, size=X_syn_df.shape[0])
X_encoded = Obj.WOE_fit(X_syn_df, y_syn_df)
X_encoded.head()

,Unnamed: 0,SeniorCitizen,tenure,MonthlyCharges,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,0.956451,0.145626,0.145626,0.145626,-1.112095,-0.366287,0.145626,-1.112095,0.145626,-0.366287,0.956451,0.145626,0.000000,-0.366287,0.568693,-0.366287,-1.112095,-0.366287,-0.366287,0.145626
1,0.956451,-0.366287,1.340932,-0.366287,-1.112095,0.145626,0.145626,0.145626,0.568693,-1.112095,-1.112095,0.568693,-1.112095,-0.366287,-0.366287,-0.366287,0.000000,-0.366287,-0.366287,-0.366287
2,0.956451,0.145626,0.956451,0.145626,0.568693,0.145626,0.956451,0.145626,0.568693,0.145626,0.568693,0.145626,0.145626,0.000000,0.956451,0.145626,-0.366287,0.568693,-0.366287,0.145626
3,0.956451,0.568693,0.956451,0.568693,1.340932,0.568693,0.568693,-1.112095,0.956451,0.145626,0.568693,0.145626,0.956451,0.956451,0.145626,-1.112095,-0.366287,0.956451,0.145626,1.752493
4,0.956451,0.956451,-0.366287,0.145626,0.956451,0.956451,0.145626,0.145626,0.568693,2.903949,0.145626,0.145626,0.568693,0.568693,0.568693,0.568693,-0.366287,-0.366287,0.145626,0.145626


In [ ]:
org_test_df=pd.read_csv('drive/My Drive/Original_data_GAN_mWOE_privacy_1_30percent_testing_dataset7043.csv')
y_test=org_test_df['churn']
X_test=org_test_df.drop(['churn'], axis=1)
Obj_test = WOE_Encoder(cols=X_test.columns, size=X_test.shape[0])
X_encoded_test = Obj_test.WOE_fit(X_test, y_test)
X_encoded_test.head()

,Unnamed: 0,SeniorCitizen,tenure,MonthlyCharges,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,0.569715,-0.155969,1.586662,-0.110356,-0.021324,-0.412361,0.229132,-0.033252,-0.033252,-0.482741,0.699000,0.610469,0.630512,0.728991,0.268615,0.367391,0.733324,0.331370,0.885785,0.967358
1,-1.172108,-0.155969,-1.191184,-1.455578,0.023052,0.318783,0.229132,0.003738,0.169368,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,0.733324,0.331370,-0.692514,-0.506478
2,0.967358,-0.155969,-1.233862,-0.259956,-0.021324,-0.412361,-0.696401,0.003738,-0.141946,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-1.613754,-2.375949,-0.650735,-0.422533,0.569715
3,0.136166,-0.155969,1.586662,0.967358,-0.021324,0.318783,0.229132,0.003738,-0.141946,0.722046,0.699000,0.610469,-0.269859,0.728991,0.268615,0.367391,0.733324,-0.650735,0.885785,-0.390795
4,0.136166,-0.155969,-1.593819,1.363492,0.023052,0.318783,0.229132,0.003738,-0.141946,-0.482741,0.699000,0.610469,0.630512,-0.831894,0.268615,0.367391,-2.375949,-0.650735,-0.692514,0.136166


In [ ]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=6.579332246575682e-09)

classifier.fit(X_encoded,y_syn_df)


cv_method = RepeatedKFold(n_splits=10,
                          n_repeats=3,
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 5)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1283 256 133 441
pod:  0.7682926829268293
pof:  0.1663417803768681
AUC:  0.8009754512749806
recall:  0.7682926829268293
precision:  0.6327116212338594
F1-Score:  0.6939417781274588
accuracy:  0.8159015617605301


In [ ]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)



1319 220 142 432
pod:  0.7526132404181185
pof:  0.142949967511371
AUC:  0.8048316364533737
recall:  0.7526132404181185
precision:  0.6625766871165644
F1-Score:  0.7047308319738989
accuracy:  0.828679602460956


In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(bootstrap= True, max_depth= 80, max_features= 3, min_samples_leaf= 3, min_samples_split= 12, n_estimators= 1000)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1428 111 279 295
pod:  0.5139372822299652
pof:  0.07212475633528265
AUC:  0.7209062629473412
recall:  0.5139372822299652
precision:  0.7266009852216748
F1-Score:  0.6020408163265306
accuracy:  0.8154283009938476


In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(metric='manhattan', weights='uniform', n_neighbors=19 )
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1379 160 188 386
pod:  0.6724738675958188
pof:  0.10396361273554255
AUC:  0.7842551274301381
recall:  0.6724738675958188
precision:  0.706959706959707
F1-Score:  0.6892857142857143
accuracy:  0.8353052531945102


In [ ]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

#classifier = DecisionTreeClassifier(criterion='entropy', splitter='best')
classifier = DecisionTreeClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1293 246 263 311
pod:  0.5418118466898955
pof:  0.15984405458089668
AUC:  0.6909838960544994
recall:  0.5418118466898955
precision:  0.5583482944344704
F1-Score:  0.5499557913351018
accuracy:  0.759110269758637


In [ ]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1430 109 277 297
pod:  0.5174216027874564
pof:  0.07082521117608837
AUC:  0.7232981958056841
recall:  0.5174216027874564
precision:  0.7315270935960592
F1-Score:  0.6061224489795918
accuracy:  0.8173213440605773


In [ ]:
#XGBClassifier
from xgboost import XGBClassifier
classifier =  XGBClassifier(random_state=0)

classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X, y = y, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1450 89 295 279
pod:  0.48606271777003485
pof:  0.057829759584145546
AUC:  0.7141164790929447
recall:  0.48606271777003485
precision:  0.7581521739130435
F1-Score:  0.5923566878980892
accuracy:  0.8182678655939423


In [ ]:
#FNN
from keras.models import Sequential
from keras.layers import Dense
#create model
model = Sequential()
#get number of columns in training data
n_cols = X_encoded_test.shape[1]
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#compile model using mse as a measure of model performance
#model.compile(optimizer='adam', loss='mean_squared_error')
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
from keras.callbacks import EarlyStopping
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)
#train model

model.fit(X_encoded, y_syn_df, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
y_pred = model.predict(X_encoded_test)
y22_pred=y_pred.round()
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp  = confusion_matrix(y_test, y22_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

Epoch 1/30
124/124 [==============================] - 5s 6ms/step - loss: 0.3387 - accuracy: 0.8545 - val_loss: 0.2854 - val_accuracy: 0.8783
Epoch 2/30
124/124 [==============================] - 1s 4ms/step - loss: 0.2450 - accuracy: 0.8940 - val_loss: 0.2703 - val_accuracy: 0.8986
Epoch 3/30
124/124 [==============================] - 1s 4ms/step - loss: 0.2252 - accuracy: 0.9064 - val_loss: 0.2483 - val_accuracy: 0.9016
Epoch 4/30
124/124 [==============================] - 1s 4ms/step - loss: 0.2112 - accuracy: 0.9168 - val_loss: 0.2463 - val_accuracy: 0.9077
Epoch 5/30
124/124 [==============================] - 1s 4ms/step - loss: 0.2019 - accuracy: 0.9158 - val_loss: 0.2561 - val_accuracy: 0.8996
Epoch 6/30
124/124 [==============================] - 1s 4ms/step - loss: 0.2004 - accuracy: 0.9204 - val_loss: 0.2435 - val_accuracy: 0.9047
Epoch 7/30
124/124 [==============================] - 1s 4ms/step - loss: 0.1901 - accuracy: 0.9249 - val_loss: 0.2504 - val_accuracy: 0.9016
Epoch 